In [65]:
import dpkt
import datetime
import itertools
from dpkt.utils import mac_to_str, inet_to_str
from collections import namedtuple


In [66]:
def drop_packet(pkt):
    # simple filer to keep the packets we are interested in
    return not hasattr(p, 'tcp')

In [71]:
class Packet:
    def __init__(self, pkt):
        timestamp, buf = pkt
        self.timestamp = timestamp
    
        # Unpack the Ethernet frame (mac src/dst, ethertype)
        self.eth = eth = dpkt.ethernet.Ethernet(buf)

        # Make sure the Ethernet data contains an IP packet
        if not isinstance(eth.data, dpkt.ip.IP):
            return

        # Now grab the data within the Ethernet frame (the IP packet)
        self.ip = ip = eth.data
        
        self.src = ip.src
        self.dst = ip.dst

        # Check for TCP in the transport layer
        if not isinstance(ip.data, dpkt.tcp.TCP):
            return

        # Set the TCP data
        self.tcp = tcp = ip.data
        
        self.sport = tcp.sport
        self.dport = tcp.dport
        self.seq = tcp.seq
        self.ack = tcp.ack
        
    def dump(self):
        
        out = str(datetime.datetime.utcfromtimestamp(self.timestamp))
        if hasattr(self, 'src'):
            out += f' {inet_to_str(self.src)}'
            
        if hasattr(self, 'sport'):
            out += f':{self.sport}'
            
        if hasattr(self, 'dst'):
            out += f' -> {inet_to_str(self.dst)}'
            
        if hasattr(self, 'dport'):
            out += f':{self.dport}'
            
        return out
    
    def key(self):
        return (self.src, self.sport, self.dst, self.dport)
    
    def reverse_key(self):
        return (self.dst, self.dport, self.src, self.sport)
           

In [68]:
class Monologue:
    def __init__(self, pkt):
        # create a new monologue instance from the Packet
        self.last_seq = pkt.seq
        self.bytes = 0
        self.pkt = pkt
        
    def update(self, pkt):
        if pkt.seq > self.last_seq:
            self.bytes += pkt.seq - self.last_seq
            self.last_seq = pkt.seq
        
        

In [70]:
with open('pcap/racedepartment-slow.pcap', 'rb') as f:
    mono = dict()
    reader = dpkt.pcap.Reader(f)
    #head = itertools.islice(reader, 5) # grab the first five elements
    for pkt in reader:
        p = Packet(pkt)
        if drop_packet(p):
            continue
            
        key = p.key()
        if key not in mono:
            mono[key] = Monologue(p)
            
        mono[key].update(p)
        

    # dump the monologues
    for m in mono.values():
        print(f'{m.pkt.dump()} {m.bytes}')
        

2022-01-14 21:35:07.591306 192.168.152.116:57368 -> 31.204.153.0:443 8577
2022-01-14 21:35:07.624399 31.204.153.0:443 -> 192.168.152.116:57368 400531555
2022-01-14 21:54:07.777341 192.168.152.116:57492 -> 31.204.153.0:443 9415
2022-01-14 21:54:07.807662 31.204.153.0:443 -> 192.168.152.116:57492 170871
